# Лабораторная работа 1

In [10]:
from dataclasses import dataclass, field

import numpy as np

In [143]:
@dataclass
class LinearCode:
    G: np.ndarray = field(init=True)
    k: int = field(init=False, default=None)
    n: int = field(init=False, default=None)
    G_REF: np.ndarray = field(init=False, default=None)
    G_RREF: np.ndarray = field(init=False, default=None)
    X: np.ndarray = field(init=False, default=None)
    I: np.ndarray = field(init=False, default=None)
    
    def __post_init__(self):
        self.I = np.eye(len(self.G))
    
    def REF(self):  #1.1
        """ Приведение матрицы к ступенчатому виду (REF) """
        A = self.G.copy()
        rows, cols = A.shape
        lead = 0
        for r in range(rows):
            if lead >= cols:
                return A
            i = r
            while A[i, lead] == 0:
                i += 1
                if i == rows:
                    i = r
                    lead += 1
                    if lead == cols:
                        return A
            A[[i, r]] = A[[r, i]] 
            A[r] = A[r] % 2  
            for i in range(r + 1, rows):
                if A[i, lead] != 0:
                    A[i] = (A[i] + A[r]) % 2  
            lead += 1
        return A
    
    def RREF(self):  #1.2
        """ Приведение матрицы к приведённому ступенчатому виду (RREF) """
        A = self.REF()
        rows, cols = A.shape
        for r in range(rows - 1, -1, -1):
            lead = np.argmax(A[r] != 0)
            if A[r, lead] != 0:
                for i in range(r):
                    A[i] = (A[i] - A[i, lead] * A[r]) % 2
        return A
    
    def get_n_k(self):  # 1.3.2
        """Возвращает размеры матрицы без учета полностью нулевых строк"""
        if self.G_REF is None:
            self.G_REF = self.REF()
        
        if not (self.k or self.n):
            self.k = np.sum(np.any(self.G_REF != 0, axis=1))
            self.n = self.G_REF.shape[1]
        
        return self.k, self.n
    
    def generate_H_matrix(self):  # 1.3.3
        """ Формирование проверочной матрицы H """
        self.G_RREF = self.RREF()
        
        lead_columns = self.get_lead_columns()
        
        k, n = self.get_n_k()
        non_lead_columns = [i for i in range(self.n) if i not in lead_columns]
        X = self.G_RREF[:, non_lead_columns]
        
        I = np.eye(len(non_lead_columns), dtype=int)
        H = np.zeros((n, len(non_lead_columns)), dtype=int)
        x_index = 0
        i_index = 0
        
        for row in range(n):
            if row in lead_columns:
                H[row, :] = X[x_index, :]
                x_index += 1
            else:
                H[row, :] = I[i_index, :]
                i_index += 1
        
        return H
    
    def get_lead_columns(self):
        """ Получение индексов ведущих столбцов """
        if self.G_RREF is None:
            self.G_RREF = self.RREF()
        lead_columns = []
        for row in self.G_RREF:
            lead_col = np.argmax(row != 0)
            if row[lead_col] == 1:
                lead_columns.append(lead_col)
        return lead_columns
    
    def generate_codewords(self):
        """ Генерация всех кодовых слов """
        codewords = []
        self.get_n_k()
        for i in range(2 ** self.k):
            u = np.array(list(np.binary_repr(i, self.k)), dtype=int)
            v = (u @ self.G) % 2
            codewords.append(v)
        return np.array(codewords)
    
    def introduce_error(self, codeword, num_errors):
        """ Внесение ошибки в кодовое слово """
        error = np.zeros_like(codeword)
        indices = np.random.choice(len(codeword), num_errors, replace=False)
        error[indices] = 1
        return (codeword + error) % 2
    
    def check_error(self, codeword_with_error):
        """ Проверка кодового слова с ошибкой на проверочной матрице H """
        H = self.generate_H_matrix()
        syndrome = (codeword_with_error @ H) % 2
        return syndrome
    
    def compute_distance(self):
        """ Вычисление кодового расстояния """
        codewords = self.generate_codewords()
        min_weight = np.inf
        for word in codewords:
            weight = np.sum(word)
            if 0 < weight < min_weight:
                min_weight = weight
        return min_weight
    
    def test_generate_H_matrix(self):
        """ Тест генерации проверочной матрицы H """
        H = self.generate_H_matrix()
        print("H matrix:\n", H)
    
    def test_codewords_and_error(self):
        """ Тест генерации кодовых слов и работы с ошибками """
        codewords = self.generate_codewords()
        print("Codewords:\n", codewords)
        
        # Внесение ошибки
        codeword = codewords[0]
        print("Original codeword:\n", codeword)
        
        codeword_with_error = self.introduce_error(codeword, 2)
        print("Codeword with error:\n", codeword_with_error)
        
        # Проверка ошибки
        syndrome = self.check_error(codeword_with_error)
        print("Syndrome:\n", syndrome)
    
    def test_distance(self):
        """ Тест вычисления кодового расстояния """
        distance = self.compute_distance()
        print(f"Code distance: {distance}")
    
    def find_error_t_plus_1(self, codeword):
        """ Найти ошибку кратности t+1, которая не может быть обнаружена """
        H = self.generate_H_matrix()
        num_errors = H.shape[1] - 1
        error = self.introduce_error(codeword, num_errors)
        syndrome = (error @ H.T) % 2
        return error, syndrome
    
    def test_error_t_plus_1(self, codeword):
        """ Тест ошибки кратности t+1, которая не может быть обнаружена """
        error, syndrome = self.find_error_t_plus_1(codeword)
        print(f"Error t+1:\n{error}")
        print(f"Syndrome:\n{syndrome}")

In [144]:
S = np.array([
    [1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
    [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
    [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
    [1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1],
    [0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0],
    [1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]
])

code = LinearCode(S)
print(S)

[[1 0 1 1 0 0 0 1 0 0 1]
 [0 0 0 1 1 1 0 1 0 1 0]
 [0 0 0 0 1 0 0 1 0 0 1]
 [1 0 1 0 1 1 1 0 0 0 1]
 [0 0 0 0 1 0 0 1 1 1 0]
 [1 0 1 1 1 0 0 0 0 0 0]]


1.3.1 На основе входной матрицы сформировать порождающую
матрицу в ступенчатом виде.


In [145]:
REF = code.REF()
print(REF)

[[1 0 1 1 0 0 0 1 0 0 1]
 [0 0 0 1 1 1 0 1 0 1 0]
 [0 0 0 0 1 0 0 1 0 0 1]
 [0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0]]


1.3.2 Задать n равное числу столбцов и k равное числу строк
полученной матрицы (без учёта полностью нулевых строк).

In [146]:
print(f'{REF}\n')
k, n = code.get_n_k()
print(f"k: {k}, n: {n}")

[[1 0 1 1 0 0 0 1 0 0 1]
 [0 0 0 1 1 1 0 1 0 1 0]
 [0 0 0 0 1 0 0 1 0 0 1]
 [0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0]]

k: 5, n: 11


1.3.3 Сформировать проверочную матрицу на основе порождающей.

In [147]:
G = np.array([
    [1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
    [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
    [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
])
code = LinearCode(G)
print(G)

[[1 0 1 1 0 0 0 1 0 0 1]
 [0 0 0 1 1 1 0 1 0 1 0]
 [0 0 0 0 1 0 0 1 0 0 1]
 [0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 1 1]]


In [148]:
H = code.generate_H_matrix()
print(H)

[[0 1 1 1 1 0]
 [1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 1 1]
 [0 0 0 1 0 1]
 [0 0 1 0 0 0]
 [0 0 0 0 1 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 1]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]


Шаг 3. Сформировать сокращённую матрицу X, удалив ведущие
столбцы матрицы G

In [149]:
H = code.generate_H_matrix()
print(H)

[[0 1 1 1 1 0]
 [1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 1 1]
 [0 0 0 1 0 1]
 [0 0 1 0 0 0]
 [0 0 0 0 1 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 1]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]


1.4. Сформировать все кодовые слова длины n двумя способами.

1.4.1 Сложить все слова из порождающего множества, оставить
неповторяющиеся.

1.4.2 Взять все двоичные слова длины k, умножить каждое на G.

In [150]:
G = np.array([
    [1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
    [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
    [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
])
code = LinearCode(G)

In [151]:
 # Тест для генерации кодовых слов и работы с ошибками
code.test_codewords_and_error()
    
# Тест для вычисления кодового расстояния
code.test_distance()
    
# Тест ошибки кратности t+1
codeword = [1, 0, 1, 1, 0, 1]
code.test_error_t_plus_1(codeword)

Codewords:
 [[0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 1 0 1 0 1]
 [0 0 0 0 1 0 0 1 0 0 1]
 [0 0 0 0 1 0 0 1 1 1 0]
 [0 0 0 0 1 0 1 1 0 1 1]
 [0 0 0 0 1 0 1 1 1 0 0]
 [0 0 0 1 1 1 0 1 0 1 0]
 [0 0 0 1 1 1 0 1 1 0 1]
 [0 0 0 1 1 1 1 1 0 0 0]
 [0 0 0 1 1 1 1 1 1 1 1]
 [0 0 0 1 0 1 0 0 0 1 1]
 [0 0 0 1 0 1 0 0 1 0 0]
 [0 0 0 1 0 1 1 0 0 0 1]
 [0 0 0 1 0 1 1 0 1 1 0]
 [1 0 1 1 0 0 0 1 0 0 1]
 [1 0 1 1 0 0 0 1 1 1 0]
 [1 0 1 1 0 0 1 1 0 1 1]
 [1 0 1 1 0 0 1 1 1 0 0]
 [1 0 1 1 1 0 0 0 0 0 0]
 [1 0 1 1 1 0 0 0 1 1 1]
 [1 0 1 1 1 0 1 0 0 1 0]
 [1 0 1 1 1 0 1 0 1 0 1]
 [1 0 1 0 1 1 0 0 0 1 1]
 [1 0 1 0 1 1 0 0 1 0 0]
 [1 0 1 0 1 1 1 0 0 0 1]
 [1 0 1 0 1 1 1 0 1 1 0]
 [1 0 1 0 0 1 0 1 0 1 0]
 [1 0 1 0 0 1 0 1 1 0 1]
 [1 0 1 0 0 1 1 1 0 0 0]
 [1 0 1 0 0 1 1 1 1 1 1]]
Original codeword:
 [0 0 0 0 0 0 0 0 0 0 0]
Codeword with error:
 [0 1 0 0 0 0 0 0 0 0 1]
Syndrome:
 [1 0 0 0 0 1]
Code distance: 2
Error t+1:
[0 1 0 0 1 1]
Syndrome:
[0 0 1 0 1 0 1 0 0 1